I needed to import several libraries apart from the ones that were provide to us at the start to properly complete this project. Here I will make note of those libraries but first I needed to download some things. Besdies wordnet, I had to download punkt and stopwords. I needed to imort several sklearn abilities as well. The WordLemmatizer allowed me to lemmatize the dataset. Train_test_split was used to split the dataframe into training data and test data for the models. TfidfVectorizer and CountVectorizer were used to vectorize the text data into tfidf and bow forms respectively. Perceptron and Logistic Regression allowed me to train, test, and evaluate the Perceptron and Logistic Regression models respectively as the names would suggest. LineSVC allowed me to do the same but for the SVM model. This package is a bit different from the initial SVM package that I initially used. I selected it because it generated better f1 scores and decreased the runtime. Lastly, for the models, I used MultinomialNB to train, test, and evaluate the Naive Bayes model. This is also differrent from the more widely used GaussianNB and like the SVM package, it too prevented crashes due to lack of RAM. 

* I would like to know that I'm not sure what implementation is optimal memory wise because I ended up paying for extra Google Colab resources just to run my code. 

Now, to actually evaluate these models I needed to import precision_score, recall_score, and f1_score which calculate the precision, recall, and f1 score of a given model. To each of these evalution functions you must pass the test labels and the prediction that your model gave. 

Finally, I imported StandardScaler which allowed me to standardize the data befor giving it to the models.

* I used Python 3.9.1

In [168]:
import pandas as pd
import numpy as np

import nltk
nltk.download('wordnet')

import re
from bs4 import BeautifulSoup

nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/buckethoop/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/buckethoop/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/buckethoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [169]:
! pip install bs4 # in case you don't have it installed
! pip install regex 
# Dataset: https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

## Read Data

For this segement I read in the dataset and indicate a tabular separator due to file being in tsv format. I get a warning indicating that column 7 has mixed types but for the aim of this project I didn't think it was necessary to address. This is because we are only dealing with the star_rating and review_body columns neither of which are column 7.

In [170]:
amazon_data = pd.read_csv('data.tsv', sep='\t', on_bad_lines='skip')

/var/folders/g2/2wlpbbpn0x7022x3r6twsj540000gn/T/ipykernel_18781/469342672.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_data = pd.read_csv('data.tsv', sep='\t', on_bad_lines='skip')


## Keep Reviews and Ratings

This code overwrites the original dataframe and changes it to a dataframe with only the star_review and review_body columns. There is also a line in the segment that drops any instance where a rating isn't given. This means that it will not appear in the new dataframe. 

In [171]:
important_fields = ['star_rating', 'review_body']
amazon_data = amazon_data[important_fields]
amazon_data.dropna(subset=['star_rating'], inplace=True) #remove reviews with no star ratings

 ## We form two classes and select 50000 reviews randomly from each class.



This segment focuses on assigning each review a class. Reviews with lower and neutral ratings are put into one class and reviews with higher ratings are put into another. Once the classes are determined a random sample size of the data is taken from both classes. These samples will then become a new dataframe which includes the random entries from class 1 and class 2. It is important to note that for this project the sample size should be 50,000 but I created the variable "review_size" so that it would be easy to change this value when doing quick test runs. 

In [172]:
def classify_ratings(ratings):
    if(ratings in [1,2,3]):
        return 1
    else:
        return 2

amazon_data['Class'] = amazon_data['star_rating'].apply(classify_ratings)

data_class1 = amazon_data[amazon_data['Class'] == 1]
data_class2 = amazon_data[amazon_data['Class'] == 2]

review_size = 50000

class1_rand = data_class1.sample(n=review_size, random_state=4)
class2_rand = data_class2.sample(n=review_size, random_state=4)

amazon_data = pd.concat([class1_rand,class2_rand], axis=0)

        star_rating                                        review_body  Class
1391604           3  I use this headset for my transcription work. ...      1
2511931         1.0  The film is too stiff and attracts every singl...      1
1050169         3.0                                               good      1
1915924           2  If you want &#34;MINI&#34; happy face stickers...      1
1724146           1  I bought this wi-fi along with obi device and ...      1
2457805         2.0  I got my mini av projector in the mail and 2 d...      1
889930            3  Works as it should, but not liking the price h...      1
485710            3                Decent envelopes.  Nothing special.      1
170257            3  The cards are definitely made with quality in ...      1
2422675         1.0  The picture that was displayed in the advertis...      1
950457            4  Bought as a gift, plus one for me. I like the ...      2
1620430           5  Anyone would find this card funny.  Its got

# Data Cleaning



# Pre-processing

This segment performs the data cleaning aspect of this program. There are several points to this that I will detail in this section. I have created a contraction dictionary, contraction_dict, that holds contractions and there expanded forms. I chose this approach because I came across a document listing some of the most common contractions and want to use that as reference. (https://www.sjsu.edu/writingcenter/docs/handouts/Contractions.pdf) This approach also gave me the control to pick which expanded form I wanted to use in the case that I contraction yielded more than one expanded form. 
Next, there is a function that calculate the average amount of characters in a dataframes column. For the purposes of this project, the function is only applied to the column that holds the reviews. This function will be called before the data cleaning process as well as after to illustrate how data cleaning can weed out items that we believe won't valuable information about the dataset. After, there is a data cleaning function that encapsulates all of the data cleaning procedures I decided to use on the dataset. In the data cleaning function there exist the code to: change reviews to lowercase and remove extra spaces on the beginning and end of the entry, expand contractions, remove punctuation, remove html, remove urls, remove a word if it is less than 3 characters and remove extra spaces. 

I chose to incorporate the removal of words that are less than 3 characters because I believed that most of these less than 3 character words would either be typos or stop words that would be removed later on. 

The next step is assuring that the reviews are in string format before using the data cleaning function. There is one more step before performing data cleaning which is getting the average lenght in characters of the reviews in the dataframe. Then we perform data cleaning and finally get the average length in characters of the reviews in the dataframe once more as the review_body column is now clean. 

* It may be important to note that the contraction expanding code does increase the average length slightly

Avg Length Before Data clean: 319.6495 
<br>
Avg Length After Data Clean: 266.33735

In [173]:
contraction_dict = {
    "won't" : "will not",
    "couldn't" : "could not",
    "shouldn't" : "should not",
    "wouldn't" : "would not",
    "aren't" : "are not",
    "didn't" : "did not",
    "can't" : "can not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "I'd" : "I would", #specifically picked this instead of I had
    "I'll" : "I will",
    "I'm" : "I am",
    "I've" : "I have",
    "isn't" : "is not",
    "let's" : "let us",
    "that's" : "that is",
    "there's": "there is",
    "they'd" : "they had",
    "they'll" : "they will",
    "weren't" : "were not",
    "what're" : "what are",
    "what's" : "what is",
    "where's" : "where is",
    "you'll" : "you will",
    "you're" : "you are",
    "it's" : "it is",
    "we're" : "we are",
    "y'all" : "you all",
    "it'll" : "it will",
    "could've" : "could have",
    "would've" : "would have",
    "should've" : "should have",
    "ain't" : "is not",
    "he'll" : "he will",
    "he's" : "he is",
    "she'll" : "she will",
    "she's" : "she is",
    "it'd" : "it would",
}

def review_avg(dataframe, column):
    dataframe['review_length'] = dataframe[column].apply(len)
    avg_length = dataframe['review_length'].mean()
    return avg_length

#generates a new column in the dataframe with cleaned reviews
def data_clean(dataframe):

    dataframe['clean_review'] = dataframe['review_body'].apply(lambda x: x.lower().strip()) #change to lowercase and remove extra spaces on beginning/end of entry
    dataframe['clean_review'] = dataframe['clean_review'].apply(lambda x: ' '.join([contraction_dict.get(contraction, contraction) for contraction in x.split()])) #expands contractions
    dataframe['clean_review'] = dataframe['clean_review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x)) #remove punctutation /does it also remove numbers?
    dataframe['clean_review'] = dataframe['clean_review'].apply(lambda x: BeautifulSoup(x,'html.parser').get_text()) #remove html
    dataframe['clean_review'] = dataframe['clean_review'].apply(lambda x: ' '.join([word for word in x.split() if len(word) >= 3]))
    dataframe['clean_review'] = dataframe['clean_review'].apply(lambda x: re.sub(r'http\S+|www\S+|https|S+','',x,flags=re.MULTILINE)) #remove urls
    dataframe['clean_review'] = dataframe['clean_review'].apply(lambda x: re.sub(' +', ' ', x)) #remove extra spaces

    return dataframe

amazon_data['review_body'] = amazon_data['review_body'].astype(str) #assure that all entries are in string format
cleaning_avg_length_before = review_avg(amazon_data,'review_body') #set the average length of the reviews before the data cleaning step
clean_data = data_clean(amazon_data)

cleaning_avg_length_after = review_avg(clean_data,'clean_review') #set the average length of the reviews after the data cleaning step

print(f"The average length of the reviews in terms of character length BEFORE & AFTER Data Cleaning: {cleaning_avg_length_before}, {cleaning_avg_length_after}")

The average length of the reviews in terms of character length BEFORE & AFTER Data Cleaning: 462.9, 377.6


## remove the stop words 

Continuing on with the preprocessing step, this segment is tasked with remove the stop words from the dataset. I chose to tokenize the contents of the review column because I felt that it would be easier to implement and more readable to pass the values the preprocessing functions were returning to the next function in line. Similar to the data cleaning section, we were asked calculate the average review length in character before and after the remove stop words and lemmatization steps. 

In [174]:
#tokenize reviews and stop words
def rmv_stopwords(review):
    split_reviews = nltk.word_tokenize(review)
    tokens = [word for word in split_reviews if word not in stopwords.words('english')]

    return ' '.join(tokens)

preprocessing_avg_length_before = review_avg(amazon_data,'clean_review') #set the average length of the reviews before the data cleaning step

amazon_data['clean_review'] = amazon_data['clean_review'].apply(rmv_stopwords)

## perform lemmatization  

In this segment you'll find the function to lemmatize the data. You will also find a function, preprocess that takes the review column and perform the previous 3 funtions on it. Lastly, we calculate the average length of the reviews in characters to show how much the preprocessing step decreases the average amount of characters. 

*I checked Piazza later on in the project and realized that there's a possibility that I didn't implement lemmatization correctly because I didn't account for POS tagging. However, due to me seeing this information later I wasn't able to implement. 

Avg Length Before Preprocessing: 266.33735
<br>
Avg Length After Preprocessing: 189.19202

In [175]:
def lemmatize_review(review):
    return [WordNetLemmatizer().lemmatize(review) for token in review]

amazon_data['clean_review'] = amazon_data['clean_review'].apply(lemmatize_review)

preprocessing_avg_length_after = review_avg(amazon_data,'clean_review') #set the average length of the reviews after the data cleaning step

print(f"The average length of the reviews in terms of character length BEFORE & AFTER Preprocessing: {preprocessing_avg_length_before}, {preprocessing_avg_length_after}")

The average length of the reviews in terms of character length BEFORE & AFTER Preprocessing: 377.6, 264.55


# Preparing Data Feature Extraction and Models

This is a section I inserted so that the section that were actually asked of us weren't too cluttered. It mainly sets up the data that will be needed for the feature extraction. It also split the data into a train set and a test set. 20% of the data will be allocated for testing as per the instructions.

In [176]:
amazon_data['clean_review'] = amazon_data['clean_review'].apply(lambda x: ' '.join(x)) #assure that all entries are in string format

reviews = amazon_data['clean_review']
labels = amazon_data['Class']

#splitting data
rev_split_train, rev_split_test, labels_train, labels_test = train_test_split(reviews, labels, test_size=.2,random_state=0)

# TF-IDF and BoW Feature Extraction

This segment handles the two types of feature extraction we were asked to perform, tf-idf and bag of word. I use CountVectorizer and TfidVectorizer from sklearn. The CountVectorizer will allow for the review data to be in a bag of word representation. The same idea is used for Tf-idf the only real different is that we use TfidVectorizer instead of Count Vectorizer. 
One thing that was interesting to me is that it is expected to use .fit_transform on the training set and only .transform on the test set. This was because .fit_trasnform vectorizes the data but also set parameters on it and learns from it. On the otherhand, for the testing set we want to apply what we learned from the training set and apply it to a test set the model hasn't seen before. 
I have also used StandardScaler to standardize the sparse matrix, the "with_mean" parameter is set to false here because we don't want to center the data of a sparse matrix.

* I would like to note that i set the max features to be extracted to 8000. Initially I had 5000 but I had better results with 8000.

In [177]:

bow_vect = CountVectorizer(max_features=8000)
bow_feat = bow_vect.fit_transform(rev_split_train)
bow_test = bow_vect.transform(rev_split_test)

ss_bow = StandardScaler(with_mean=False)
bow_feat_ss = ss_bow.fit_transform(bow_feat)
bow_test_ss = ss_bow.transform(bow_test)

tfidf_vect = TfidfVectorizer(max_features=8000)
tfidf_feat = tfidf_vect.fit_transform(rev_split_train)
tfidf_test = tfidf_vect.transform(rev_split_test)

ss_tfidf = StandardScaler(with_mean=False)
tfidf_feat_ss = ss_tfidf.fit_transform(tfidf_feat)
tfidf_test_ss = ss_tfidf.transform(tfidf_test)

# Perceptron Using Both Features

This segment, as well as the segments that follow deal with training, testing, and evaluation each models performance with bow and tf-idf. Here were are training two Perceptron instances. The model is trained with standarized bow features. Once the model is trained, it makes a prediction on the test set. We then use different methods of evaluation to discover how well the model learns. We then repeat the same steps for tf-idf features.

BOW Prec: 0.772786263352301 ,Rec: 0.7735585090436694 , F1: 0.7731721933679584 
<br>
TFIDF Prec: 0.7817998994469583 ,Rec: 0.776956130708504 , F1: 0.7793704891740175 

In [178]:
perceptron = Perceptron(max_iter=1000)
perceptron.fit(bow_feat_ss, labels_train)
bow_predict = perceptron.predict(bow_test_ss)

#evaluate bow
per_bow_precision = precision_score(labels_test,bow_predict)
per_bow_recall = recall_score(labels_test,bow_predict)
per_bow_f1 = f1_score(labels_test,bow_predict)

print(f"Precision: {per_bow_precision}, Recall: {per_bow_recall} , F1 score: {per_bow_f1} ")

perceptron = Perceptron(max_iter=1000)
perceptron.fit(tfidf_feat_ss, labels_train)
tfidf_predict = perceptron.predict(tfidf_test_ss)

#evaluate tf idf
per_tfidf_precision = precision_score(labels_test,tfidf_predict)
per_tfidf_recall = recall_score(labels_test,tfidf_predict)
per_tfidf_f1 = f1_score(labels_test,tfidf_predict)

print(f"Precision: {per_tfidf_precision}, Recall: {per_tfidf_recall}, F1 score: {per_tfidf_f1} ")

Precision: 0.0, Recall: 0.0 , F1 score: 0.0 
Precision: 0.0, Recall: 0.0, F1 score: 0.0 


# SVM Using Both Features

This segment, as well as the segments that follow deal with training, testing, and evaluation each models performance with bow and tf-idf. Here were are training two SVM instances. The model is trained with standarized bow features. Once the model is trained, it makes a prediction on the test set. We then use different methods of evaluation to discover how well the model learns. We then repeat the same steps for tf-idf features. 
I went with a larger number of max iterations for this model because I was receiving a warning that the model wasn't converging. 

Prec: 0.8315355714458853 ,Rec: 0.6856200659538323 , F1: 0.7515609595793623 
<br>
Prec: 0.8193973258830572 ,Rec: 0.8206255621065255 , F1: 0.8200109840730939 

In [179]:
svm = LinearSVC(max_iter=100000)
svm.fit(bow_feat_ss, labels_train)
bow_predict = svm.predict(bow_test_ss)

#evaluate bow
svm_bow_precision = precision_score(labels_test,bow_predict)
svm_bow_recall = recall_score(labels_test,bow_predict)
svm_bow_f1 = f1_score(labels_test,bow_predict)

print(f"Precision: {svm_bow_precision}, Recall: {svm_bow_recall} , F1 score: {svm_bow_f1} ")

svm = LinearSVC(max_iter=100000)
svm.fit(tfidf_feat_ss, labels_train)
tfidf_predict = svm.predict(tfidf_test_ss)

#evaluate tf idf
svm_tfidf_precision = precision_score(labels_test,tfidf_predict)
svm_tfidf_recall = recall_score(labels_test,tfidf_predict)
svm_tfidf_f1 = f1_score(labels_test,tfidf_predict)

print(f"Precision: {svm_tfidf_precision}, Recall: {svm_tfidf_recall}, F1 score: {svm_tfidf_f1} ")

Precisio: 0.3333333333333333, Recall: 0.5 , F1 score: 0.4 
Precision: 0.0, Recall: 0.0, F1 score: 0.0 


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


# Logistic Regression Using Both Features

This segment, as well as the segments that follow deal with training, testing, and evaluation each models performance with bow and tf-idf. Here were are training two Logistic Regression instances. The model is trained with standarized bow features. Once the model is trained, it makes a prediction on the test set. We then use different methods of evaluation to discover how well the model learns. We then repeat the same steps for tf-idf features. 

Prec: 0.8395491066075069 ,Rec: 0.6996102728090337 , F1: 0.7632181401940477 
<br>
Prec: 0.8201158609668397 ,Rec: 0.8205256320575597 , F1: 0.8203206953394274

In [180]:
logistic = LogisticRegression(max_iter=3000)
logistic.fit(bow_feat_ss, labels_train)
bow_predict = logistic.predict(bow_test_ss)

#evaluate bow
log_bow_precision = precision_score(labels_test,bow_predict)
log_bow_recall = recall_score(labels_test,bow_predict)
log_bow_f1 = f1_score(labels_test,bow_predict)

print(f"Precision: {log_bow_precision}, Recall: {log_bow_recall} ,F1 score: {log_bow_f1} ")

logistic = LogisticRegression(max_iter=3000)
logistic.fit(tfidf_feat_ss, labels_train)
tfidf_predict = logistic.predict(tfidf_test_ss)

#evaluate tf idf
log_tfidf_precision = precision_score(labels_test,tfidf_predict)
log_tfidf_recall = recall_score(labels_test,tfidf_predict)
log_tfidf_f1 = f1_score(labels_test,tfidf_predict)

print(f"Precision: {log_tfidf_precision}, Recall: {log_tfidf_recall}, F1 score: {log_tfidf_f1} ")

Precision: 0.3333333333333333, Recall: 0.5 ,F1 score: 0.4 
Precision: 0.0, Recall: 0.0, F1 score: 0.0 


# Naive Bayes Using Both Features

This segment, as well as the segments that follow deal with training, testing, and evaluation each models performance with bow and tf-idf. Here were are training two Naive Bayes instances. The model is trained with standarized bow features. Once the model is trained, it makes a prediction on the test set. We then use different methods of evaluation to discover how well the model learns. We then repeat the same steps for tf-idf features. 

Prec: 0.7298513711534436 ,Rec: 0.6968122314379934 , F1: 0.712949235724145 
<br>
Prec: 0.7984661269706008 ,Rec: 0.7490756470470671 , F1: 0.7729827275070895

In [181]:
naive = MultinomialNB()
naive.fit(bow_feat_ss, labels_train)
bow_predict = naive.predict(bow_test_ss)

#evaluate bow
naive_bow_precision = precision_score(labels_test,bow_predict)
naive_bow_recall = recall_score(labels_test,bow_predict)
naive_bow_f1 = f1_score(labels_test,bow_predict)

print(f"Precision: {naive_bow_precision} , Recall: {naive_bow_recall} , F1 score: {naive_bow_f1} ")

naive = MultinomialNB()
naive.fit(tfidf_feat_ss, labels_train)
tfidf_predict = naive.predict(tfidf_test_ss)

#evaluate tf idf
naive_tfidf_precision = precision_score(labels_test,tfidf_predict)
naive_tfidf_recall = recall_score(labels_test,tfidf_predict)
naive_tfidf_f1 = f1_score(labels_test,tfidf_predict)

print(f"Precision: {naive_tfidf_precision}, Recall: {naive_tfidf_recall} , F1 score: {naive_tfidf_f1} ")

Precision: 0.5 , Recall: 1.0 , F1 score: 0.6666666666666666 
Precision: 0.0, Recall: 0.0 , F1 score: 0.0 
